In [4]:
import time
import configparser
import csv
import os

from utility.m21_utility import *

from segmentation import *
from identification.key_identification import (
    determine_key_by_adjacent,
    determine_key_solo,
)
from identification.note_to_chord import find_chords
from identification.result_combination import *


directory = "../data/"
KeyThenChordMode = False

In [5]:
def get_files(filename=None):
    all_scores = [f for f in os.listdir(directory) if os.path.isfile(directory + f)]
    all_scores.remove(
        "Beethoven_L.V._Sonatina_in_A-Flat_Major_(Op.110_No.31)_2nd_Movement.mxl"
    )
    # RuntimeWarning: invalid value encountered in double_scalars correlation_value = corr_value_numerator / corr_value_denominator
    if filename in all_scores:
        return [filename]
    return all_scores

In [6]:
stream = load_file(directory + 'anonymous_Twinkle_Twinkle.mxl')
chordify_stream = chordify(stream)
flatten_stream = flatten(stream)

time_signature = get_initial_time_signature(flatten_stream)
# key_signature = get_initial_key_signature(flatten_stream) # Scale(key_signature.tonic.name)

measures_key = determine_key_by_adjacent(key_segmentation(stream))
# measures_key = determine_key_solo(key_segmentation(stream))

keys = measures_key

notes_in_measures = get_notes_in_measures(chordify_stream)
segments = uniform_segmentation(notes_in_measures, time_signature)
combined_segments = merge_chord_segment(segments)

In [7]:
combined_segments

[(0.0, {'C': 0.25, 'E': 0.25, 'G': 0.5}),
 (4.0, {'C': 0.25, 'F': 0.25, 'A': 0.5}),
 (6.0, {'C': 0.25, 'E': 0.25, 'G': 0.5}),
 (8.0, {'B': 0.2, 'C': 0.2, 'D': 0.2, 'G': 0.2, 'F': 0.2}),
 (9.0, {'B': 0.25, 'D': 0.25, 'G': 0.25, 'F': 0.25}),
 (10.0, {'C': 0.25, 'E': 0.5, 'G': 0.25}),
 (12.0, {'B': 0.2, 'C': 0.2, 'D': 0.4, 'G': 0.2}),
 (14.0, {'C': 0.25, 'E': 0.25, 'G': 0.5}),
 (18.0, {'C': 0.2, 'D': 0.2, 'G': 0.2, 'B': 0.2, 'F': 0.2}),
 (19.0, {'D': 0.25, 'G': 0.25, 'B': 0.25, 'F': 0.25}),
 (20.0, {'C': 0.25, 'E': 0.5, 'G': 0.25}),
 (22.0, {'B': 0.2, 'C': 0.2, 'F': 0.2, 'G': 0.2, 'D': 0.2}),
 (24.0, {'C': 0.25, 'E': 0.25, 'G': 0.5}),
 (26.0, {'C': 0.2, 'D': 0.2, 'G': 0.2, 'B': 0.2, 'F': 0.2}),
 (27.0, {'D': 0.25, 'G': 0.25, 'B': 0.25, 'F': 0.25}),
 (28.0, {'C': 0.25, 'E': 0.5, 'G': 0.25}),
 (30.0, {'B': 0.2, 'C': 0.1, 'F': 0.2, 'G': 0.2, 'D': 0.2, 'E': 0.1}),
 (36.0, {'C': 0.25, 'F': 0.25, 'A': 0.5}),
 (38.0, {'C': 0.4, 'E': 0.2, 'G': 0.4}),
 (40.0, {'C': 0.2, 'D': 0.2, 'G': 0.2, 'B': 0.

In [8]:
key_seg = key_segmentation(stream)
for x, y in key_seg.items():
    print(x, y)

1 {'chroma': array([6., 0., 0., 0., 4., 0., 0., 6., 0., 0., 0., 0.]), 'offset': 0.0}
2 {'chroma': array([4., 0., 0., 0., 2., 2., 0., 4., 0., 4., 0., 0.]), 'offset': 4.0}
3 {'chroma': array([2., 0., 2., 0., 4., 2., 0., 4., 0., 0., 0., 2.]), 'offset': 8.0}
4 {'chroma': array([4., 0., 4., 0., 2., 0., 0., 4., 0., 0., 0., 2.]), 'offset': 12.0}
5 {'chroma': array([2., 0., 2., 0., 2., 2., 0., 6., 0., 0., 0., 2.]), 'offset': 16.0}
6 {'chroma': array([2., 0., 2., 0., 4., 2., 0., 4., 0., 0., 0., 2.]), 'offset': 20.0}
7 {'chroma': array([2., 0., 2., 0., 2., 2., 0., 6., 0., 0., 0., 2.]), 'offset': 24.0}
8 {'chroma': array([2., 0., 2., 0., 4., 2., 0., 4., 0., 0., 0., 2.]), 'offset': 28.0}
9 {'chroma': array([6., 0., 0., 0., 4., 0., 0., 6., 0., 0., 0., 0.]), 'offset': 32.0}
10 {'chroma': array([4., 0., 0., 0., 2., 2., 0., 4., 0., 4., 0., 0.]), 'offset': 36.0}
11 {'chroma': array([2., 0., 2., 0., 4., 2., 0., 4., 0., 0., 0., 2.]), 'offset': 40.0}
12 {'chroma': array([2., 0., 1., 0., 3., 3., 0., 3., 0.

In [9]:
for x in measures_key:
    print(x)

{'offset': 0.0, 'corr_values': [((0, True), 0.967732975147163)]}
{'offset': 4.0, 'corr_values': [((5, True), 0.9170369479412213), ((0, True), 0.9755750730850131)]}
{'offset': 8.0, 'corr_values': [((0, True), 0.9755750730850131)]}
{'offset': 12.0, 'corr_values': [((0, True), 0.9755750730850131), ((7, True), 0.8630344896018174)]}
{'offset': 16.0, 'corr_values': [((0, True), 0.9755750730850131), ((7, True), 0.8630344896018174)]}
{'offset': 20.0, 'corr_values': [((0, True), 0.9588135769828824), ((7, True), 0.8219699838770042)]}
{'offset': 24.0, 'corr_values': [((0, True), 0.9610837317291874), ((7, True), 0.8219699838770042)]}
{'offset': 28.0, 'corr_values': [((0, True), 0.9692770403169964), ((7, True), 0.7469974138931429)]}
{'offset': 32.0, 'corr_values': [((0, True), 0.9692770403169964)]}
{'offset': 36.0, 'corr_values': [((5, True), 0.9170369479412213), ((0, True), 0.9692770403169964)]}
{'offset': 40.0, 'corr_values': [((0, True), 0.9692770403169964), ((2, False), 0.8921429252127325), ((5

In [10]:
res = []
for segment in combined_segments:
    notes_frequencies = [
        (note_input_convertor(note_name), v)
        for note_name, v in segment[1].items()
    ]
    key_choices = find_scale_in_chord_segment(keys, segment)
    possible_key = max(key_choices, key=key_choices.get)
    res.append(
        (segment[0], find_chords(notes_frequencies, possible_key))
    )

In [11]:
for segment in combined_segments:
    notes_frequencies = [
        (note_input_convertor(note_name), v)
        for note_name, v in segment[1].items()
    ]
    key_choices = find_scale_in_chord_segment(keys, segment)
    possible_key = max(key_choices, key=key_choices.get)
    print(find_chords(notes_frequencies, possible_key))

[({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4f8edad60>}, 1.0), ({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4f8edac40>}, 0.5), ({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4f8edaf40>}, 0.75), ({'chord': 'III', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eef400>}, 0.75)]
[({'chord': 'IV', 'scale': <preprocess.scale.Scale object at 0x7fc4f8eda730>}, 1.0), ({'chord': 'VI', 'scale': <preprocess.scale.Scale object at 0x7fc4f8edaf40>}, 0.75), ({'chord': 'IV', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eeff40>}, 0.75), ({'chord': 'IV', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eef040>}, 0.5)]
[({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4e0f4bf70>}, 1.0), ({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eef040>}, 0.5), ({'chord': 'I', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eef130>}, 0.75), ({'chord': 'III', 'scale': <preprocess.scale.Scale object at 0x7fc4e0eefd

In [12]:
key_choices

{(5, True): 0.943053279945115, (10, True): 0.7989840088530141}

In [13]:
possible_key

(5, True)

In [14]:
res

[(0.0,
  [({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4e8404520>}, 1.0),
   ({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4e8404fa0>}, 0.5),
   ({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4f8eb89d0>}, 0.75),
   ({'chord': 'III', 'scale': <preprocess.scale.Scale at 0x7fc4e856a790>},
    0.75)]),
 (4.0,
  [({'chord': 'IV', 'scale': <preprocess.scale.Scale at 0x7fc4e8404d00>}, 1.0),
   ({'chord': 'VI', 'scale': <preprocess.scale.Scale at 0x7fc4e83bbd30>},
    0.75),
   ({'chord': 'IV', 'scale': <preprocess.scale.Scale at 0x7fc4e858caf0>},
    0.75),
   ({'chord': 'IV', 'scale': <preprocess.scale.Scale at 0x7fc4d87e04f0>},
    0.5)]),
 (6.0,
  [({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4e8416970>}, 1.0),
   ({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4e84168b0>}, 0.5),
   ({'chord': 'I', 'scale': <preprocess.scale.Scale at 0x7fc4e8416c70>}, 0.75),
   ({'chord': 'III', 'scale': <preprocess.scale.Scale at 0x7fc4f878ed60>},
    

In [15]:
for offs in res:
    for tu in offs[1]:
        # print(tu)
        print(offs[0], tu[0]['scale'].scale_str())

res[0][1][0][0]['scale'].scale_str()

0.0 C Major
0.0 C Major
0.0 C Major
0.0 C Major
4.0 C Major
4.0 C Major
4.0 C Major
4.0 C Major
6.0 C Major
6.0 C Major
6.0 C Major
6.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
8.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
9.0 C Major
10.0 C Major
10.0 C Major
10.0 C Major
10.0 C Major
12.0 C Major
12.0 C Major
12.0 C Major
12.0 C Major
12.0 C Major
14.0 C Major
14.0 C Major
14.0 C Major
14.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
18.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
19.0 C Major
20.0 C Major
20.0 C Major
20.0 C Major
20.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
22.0 C Major
24.0 C Major
24.0 C Major
24.0 C Major
24.0 C Major
26.

'C Major'

In [16]:
out_list = []
for offs in res:
    out_list.append((offs[0], offs[1][0][0]['scale'].scale_str()))

In [17]:
out_list

[(0.0, 'C Major'),
 (4.0, 'C Major'),
 (6.0, 'C Major'),
 (8.0, 'C Major'),
 (9.0, 'C Major'),
 (10.0, 'C Major'),
 (12.0, 'C Major'),
 (14.0, 'C Major'),
 (18.0, 'C Major'),
 (19.0, 'C Major'),
 (20.0, 'C Major'),
 (22.0, 'C Major'),
 (24.0, 'C Major'),
 (26.0, 'C Major'),
 (27.0, 'C Major'),
 (28.0, 'C Major'),
 (30.0, 'C Major'),
 (36.0, 'C Major'),
 (38.0, 'C Major'),
 (40.0, 'C Major'),
 (41.0, 'C Major'),
 (42.0, 'C Major'),
 (44.0, 'F Major'),
 (45.0, 'F Major'),
 (46.0, 'F Major'),
 (47.0, 'F Major'),
 (48.0, 'F Major'),
 (52.0, 'F Major'),
 (54.0, 'F Major'),
 (56.0, 'F Major'),
 (58.0, 'F Major'),
 (60.0, 'F Major'),
 (62.0, 'F Major'),
 (66.0, 'F Major'),
 (68.0, 'F Major'),
 (70.0, 'F Major'),
 (72.0, 'F Major'),
 (74.0, 'F Major'),
 (76.0, 'F Major'),
 (78.0, 'F Major'),
 (80.0, 'F Major'),
 (84.0, 'F Major'),
 (86.0, 'F Major'),
 (88.0, 'F Major'),
 (90.0, 'F Major'),
 (92.0, 'F Major'),
 (94.0, 'F Major')]

In [18]:
from segmentation import *

In [19]:
from utility.m21_utility import *

from segmentation import *
from identification.key_identification import (
    determine_key_by_adjacent,
    determine_key_solo,
)
from identification.note_to_chord import find_chords
from identification.result_combination import *

In [20]:
stream = load_file('../data/anonymous_Twinkle_Twinkle.mxl')

In [21]:
for i in range(10):
    chordify_stream[i].show('text')
    # flatten_stream[i].show('text')

<music21.instrument.Piano 'P1: Piano: Piano'>
{0.0} <music21.layout.SystemLayout>
{0.0} <music21.clef.TrebleClef>
{0.0} <music21.key.KeySignature of no sharps or flats>
{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.chord.Chord C3 E3 G3 C4>
{1.0} <music21.chord.Chord C3 E3 G3 C4>
{2.0} <music21.chord.Chord C3 E3 G3 G4>
{3.0} <music21.chord.Chord C3 E3 G3 G4>
{0.0} <music21.chord.Chord C3 F3 A3 A4>
{1.0} <music21.chord.Chord C3 F3 A3 A4>
{2.0} <music21.chord.Chord C3 E3 G3 G4>
{0.0} <music21.chord.Chord B2 C3 D3 G3 F4>
{1.0} <music21.chord.Chord B2 D3 G3 F4>
{2.0} <music21.chord.Chord C3 E3 G3 E4>
{3.0} <music21.chord.Chord C3 E3 G3 E4>
{0.0} <music21.chord.Chord B2 C3 D3 G3 D4>
{1.0} <music21.chord.Chord B2 D3 G3 D4>
{2.0} <music21.chord.Chord C3 E3 G3 C4>
{0.0} <music21.chord.Chord C3 E3 G3 C4 G4>
{1.0} <music21.chord.Chord E3 G3 C4 G4>
{2.0} <music21.chord.Chord C3 D3 G3 B3 F4>
{3.0} <music21.chord.Chord D3 G3 B3 F4>
{0.0} <music21.chord.Chord C3 E3 G3 E4>
{1.0} <music21.chor

In [22]:
stream.measure(stream.duration.quarterLength)

TypeError: 'float' object cannot be interpreted as an integer

In [24]:
s = converter.parse('../data/Chopin_F._Nocturne_in_F_Minor,_Op.55_No.1.mxl')

In [38]:
fs = s.chordify()
fs.show('text')

{0.0} <music21.metadata.Metadata object at 0x7fc5090ce0a0>
{0.0} <music21.instrument.Piano 'P1: Piano: Piano'>
{0.0} <music21.stream.Measure 0 offset=0.0>
    {0.0} <music21.expressions.TextExpression 'Andante'>
    {0.0} <music21.layout.SystemLayout>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.tempo.MetronomeMark Quarter=96.0>
    {0.0} <music21.key.KeySignature of 4 flats>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.dynamics.Dynamic p>
    {0.0} <music21.chord.Chord C5>
{0.0} <music21.dynamics.Diminuendo <music21.note.Note C>>
{0.0} <music21.spanner.Slur <music21.note.Note C><music21.note.Note C>>
{0.0} <music21.spanner.Slur <music21.note.Note F><music21.note.Note C>>
{0.0} <music21.spanner.Slur <music21.note.Note F><music21.note.Note F>>
{0.0} <music21.spanner.Slur <music21.note.Note A-><music21.note.Note F>>
{0.0} <music21.spanner.Slur <music21.note.Note C><music21.note.Note C>>
{0.0} <music21.spanner.Slur <music21.note.Note F><music21.note.Note C>>
{0

In [35]:
for note in fs.getElementsByClass('Note'):
    print(note.offset, note.lyric)

0.0 None
1.0 Fm(I)
1.0 None
2.0 None
3.0 VII
3.0 None
4.0 None
5.0 III
5.0 None
5.75 None
6.0 None
6.75 None
7.0 V+7
7.0 None
9.0 I
9.0 None
10.0 None
11.0 VII
11.0 None
12.0 None
13.0 III
13.0 None
13.75 None
14.0 None
14.75 None
14.75 None
15.0 V+7
15.0 None
17.0 I
17.0 None
18.0 None
19.0 VII
19.0 None
20.0 None
21.0 III
64/3 None
23.0 None
139/6 None
23.25 None
70/3 None
70/3 None
281/12 None
23.5 None
283/12 None
143/6 None
24.0 ♭II
24.75 None
25.375 I
25.375 None
27.375 V+
27.375 None
28.375 None
29.125 None
29.375 None
29.375 I
29.375 None
31.375 None
31.375 None
32.375 V+
33.375 I
33.375 None
34.375 None
35.375 VII
35.375 None
36.375 None
37.375 I
37.375 None
38.125 None
38.375 None
39.125 None
39.375 V+7
39.375 None
41.375 I
41.375 None
42.375 None
43.375 VII
43.375 None
44.375 None
45.375 III
45.375 None
46.125 None
46.375 None
47.125 None
47.125 None
47.375 V+7
47.375 None
49.375 I
49.375 None
50.375 None
51.375 VII
51.375 None
52.375 None
53.375 III
5125/96 None
54.875 None

In [30]:
from music21 import stream as st
def get_elements_by_offset(filter=None):
    """Obtain all/specified elements in flatterned stream, ordered by offset

    :param filter: filter type based on m21 classes, defaults to None
    :type filter: str/m21 class, optional
    :return: dictionary of offsets and lists of m21 objects in {<offset>: [<element>]}
    :rtype: dict
    """
    o_iter = st.iterator.OffsetIterator(s.flatten())
    if filter:
        o_iter = o_iter.getElementsByClass(filter)
    return {x[0].offset: x for x in o_iter}

notes = get_elements_by_offset(filter="Note")
for offset, el in notes.items():
    note = el[0]
    if note.lyric and '(' in note.lyric:
        print((note, note.offset, note.lyric.split('(')[0]))

(<music21.note.Note C>, 1.0, 'Fm')
(<music21.note.Note B->, Fraction(6277, 96), 'A♭M')
(<music21.note.Note C>, Fraction(6949, 96), 'Cm')
(<music21.note.Note E>, Fraction(7429, 96), 'Fm')
(<music21.note.Note A->, Fraction(12325, 96), 'A♭M')
(<music21.note.Note C>, Fraction(13093, 96), 'Cm')
(<music21.note.Note D->, Fraction(36063, 160), 'Fm')
(<music21.note.Note E>, Fraction(45503, 160), 'Am')
(<music21.note.Note F>, Fraction(45823, 160), 'Fm')
(<music21.note.Note G->, Fraction(54303, 160), 'B♭m')
(<music21.note.Note D->, Fraction(55903, 160), 'FM')


In [ ]:
parts = stream.getElementsByClass("PartStaff")
parts = [list(part.getElementsByClass('Measure')) for part in parts]

res = []
for measures in parts:
    notes = []
    for measure in measures:
        for note in measure.getElementsByClass('Chord'):
            notes.append(note)
    res.append(notes)
res
# parts = [list(part.getElementsByClass('Measure')) for part in stream.getElementsByClass("PartStaff")]
# ms
# [[list(measure.getElementsByClass('Note')) for measure in part] for part in parts]
# [[[n for n in m.getElementsByClass('Note')] for m in part.getElementsByClass("Measure")] for part in stream.getElementsByClass("PartStaff")]
# [[measure.getElementsByClass('note') for measure in part] for part in stream.getElementsByClass("PartStaff")]

# for x in stream.recurse():
#     print(x)

In [ ]:
from music21 import stream as m21stream
# list(stream.recurse().getElementsByClass('ChordSymbol'))
# res = {x[0].offset: x for x in m21stream.iterator.OffsetIterator(flatten_stream).getElementsByClass('Note')}
# res
    # print(x[0].offset, x)
# chordify_stream.show('text')
# def __get_elements_in_measures(measures, type):
#     res = []
#     for measure in measures:
#         for el in measure.getElementsByClass(type):
#             res.append(el)
#     return res

# for part in stream.getElementsByClass("PartStaff"):
o_iter = m21stream.iterator.OffsetIterator(flatten_stream)
for x in o_iter:
    o_iter = o_iter.getElementsByClass('ChordSymbol')
cs = {x[0].offset: x for x in o_iter}

In [ ]:
for o, m in cs.items():
    for c in m:
        print(o, c.figure, c.chordKindStr)

In [ ]:
for el in stream.recurse().getElementsByClass('Note'):
    print(el.offset, el, el.activeSite)

In [ ]:
for x in stream.recurse().pitches:
    print(x)

In [ ]:
dir(stream.recurse().getElementsByClass("ChordSymbol")[0])

In [ ]:
for m in stream.recurse().getElementsByClass("pitches"):
    print(m.offset, m)

In [ ]:
for i in range(int(flatten_stream.quarterLength)):
    print(flatten_stream.getElementByOffset(i))

In [ ]:
stream.show('text')

In [ ]:
chordify_stream = chordify(stream)
flatten_stream = flatten(stream)

time_signature = get_initial_time_signature(flatten_stream)
# key_signature = get_initial_key_signature(flatten_stream) # Scale(key_signature.tonic.name)

measures_key = determine_key_by_adjacent(key_segmentation(stream))

In [ ]:
stream.show()

In [ ]:
flatten_stream.show()

In [ ]:
chordify_stream.show()

In [ ]:
res = key_segmentation(stream)

In [ ]:
res

In [ ]:
len(res[1]['chroma'])

In [ ]:
notes_in_measures = get_notes_in_measures(chordify_stream)

In [ ]:
notes_in_measures

In [ ]:
segments = uniform_segmentation(notes_in_measures, time_signature)

In [ ]:
segments

In [ ]:
measures_key